# Algoritmo para gravar o SESA com o ReSpeaker

Para que a acurácia do sistema em tempo real seja maior, vamos treinar o classificador com áudios gravados pelo próprio respeaker, assim, a etapa de teste ficará semelhante à de treinamento.

Vou analisar o tamanho do SESA_v2_16_kHz_16 bits e mandar o respeaker gravar um áudio grandão com esse tamanho. Depois disso eu mando esse áudio pro PC e separo o que é sirene, tiro, explosão e casual manualmente. Desse áudio grandão com o dataset inteiro dentro, vou separa-lo em 4. Um áudio com todos os sons de tiro, um com todos os sons de explosões, e assim por diante. O janelamento será feito posteriormente pela extração de features, e não importa muito se a gnt vai perder a informação de qual é o áudio original. 

Explicando melhor: na abordagem anterior, o CSV ficava com n janelas do áudio siren_034.wav, m janelas do áudio siren_035.wav e assim por diante. A partir de agora, ele vai ter muuuitas janelas de um mesmo arquivo, o siren_000.wav, pois esse novo arquivo siren_000.wav vai conter dentro dele todos os outros arquivos de sirene, como os próprios siren_034.wav e siren_035.wav.

O algoritmo desse jupyter pode ser rodado toda vez que o dataset for ser gravado COM O RESPEAKER. Uma abordagem de data augmentation pode envolver gravar o dataset inteiro com alto falantes diferentes, com a raspberry posicionada em lugares diferentes e tal... Tem que ver.

E vamos gravar já com os áudios sem silêncio. No PC os arquivos WAV já passaram por um processo de remoção de silêncio.

Pra fazer a gravação eu vou colocar os áudios no meu PC msm, mandar pra um alto falante top e ai é só conectar esse jupyter aqui pela raspberry e rodar ele.

### Importações

In [1]:
import pyaudio
import wave
import librosa
import os
import numpy as np
from time import time
from IPython.display import Audio

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


### Verificando o ID do dispositivo

In [2]:
def listarDispositivos():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')

    for i in range(0, numdevices):
            if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
                print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

In [3]:
listarDispositivos()

Input Device id  0  -  seeed-4mic-voicecard: bcm2835-i2s-ac10x-codec0 ac10x-codec.1-003b-0 (hw:1,0)
Input Device id  3  -  ac108


### Tempo de gravação

Só pra gnt ver qual é o tempo que tenho que deixar o respeaker gravando.

In [4]:
caminhoArquivosWAV = "/home/pi/Datasets/ESC_3_original/"

tempoTotal = 0
for audioAtual in os.listdir(caminhoArquivosWAV):
    tempoTotal += librosa.get_duration(filename=caminhoArquivosWAV+audioAtual)
    
print("Tempo (s):", tempoTotal)
print("Tempo (m):", np.array(tempoTotal)/60)

Tempo (s): 74.89569160997735
Tempo (m): 1.2482615268329558


### Gravando

In [5]:
# PARAMETROS INICIAIS 
respeakerIndex          = 0
respeakerFreqAmostragem = 16000
respeakerProfundidade   = 2
respeakerCanais         = 1
tempoGravacaoSegundos   = 90
    
# INSTANCIANDO UM OBJ PY AUDIO E MANDANDO OS PARAMETROS INICIAIS
objPyAudio = pyaudio.PyAudio()
stream = objPyAudio.open(
    input_device_index = respeakerIndex,
    rate               = respeakerFreqAmostragem,
    format             = objPyAudio.get_format_from_width(respeakerProfundidade),
    channels           = respeakerCanais,
    input=True    
)
 
# GRANVANDO
print("INICIANDO A GRAVAÇÃO")
gravacaoBinario = stream.read(tempoGravacaoSegundos * respeakerFreqAmostragem)    
print("GRAVAÇÃO FINALIZADA")
 
# MATANDO OS OBJETOS PRA LIMPAR MEMORIA
stream.stop_stream()
stream.close()
objPyAudio.terminate()

INICIANDO A GRAVAÇÃO
GRAVAÇÃO FINALIZADA


In [8]:
nomeArquivoWAVSaida.split("/")[:-1]

['', 'home', 'pi', 'Datasets', 'ESC_3_ReSpeaker_1595612055.7909768']

In [9]:
# SALVANDO O ARQUIVO WAV POIS O ARRAY JANELAS E BINARIO, NAO DA PRA OUVIR DIRETO DELE
pastaOndeSalvar     = "/home/pi/Datasets/ESC_3_ReSpeaker_"+str(time())+"/"
nomeArquivoWAVSaida = pastaOndeSalvar+"gravacaoCompleta.wav"
try:
    os.mkdir(pastaOndeSalvar)
except:
    pass

wf = wave.open(nomeArquivoWAVSaida, 'wb')
wf.setnchannels(respeakerCanais)
wf.setsampwidth(objPyAudio.get_sample_size(objPyAudio.get_format_from_width(respeakerProfundidade)))
wf.setframerate(respeakerFreqAmostragem)
wf.writeframes(b''.join([gravacaoBinario]))
wf.close()

In [10]:
# OUVINDO
sinal, freqAmostragem = librosa.load(nomeArquivoWAVSaida, sr=None, mono=False)
print(sinal.shape)
print(freqAmostragem)
Audio(data=sinal, rate=freqAmostragem)

(1440000,)
16000
